In [1]:
import pandas as pd
import numpy as np
import astropy.units as u
import astropy.constants as c
import matplotlib.pyplot as plt

import scipy
import scipy.integrate
from scipy.misc import derivative
from scipy import integrate 

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter

# Just for the Window Pop Out thing for WINDOWS
# %matplotlib qt

<B>Revised Data From Kepler 11</B>

In [2]:
K11Data = pd.read_csv('Kepler11Data.csv', header=None)

<b>Properties Needed From CSV:</b>  
Orbital Period (Days)   
Orbital Semi-Major Axis (au)  
Planet Radius (Earth radii)  
Planet Mass (Earth mass)  
Orbit Eccentricity (NO UNITS)  
Stellar Mass (Solar Mass)

--------------
<b>Probably Not Useful (all values are the same)</b>
  
Distance fom earth (Parsecs)  
RA (degrees)  
Dec (degrees) n

--------------------------------------------------------------

In [3]:
K11Data

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,pl_name,hostname,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,pl_controv_flag,pl_orbper,pl_orbpererr1,...,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2
1,Kepler-11 b,Kepler-11,1,6,Transit,2010,Kepler,0,10.30390000,0.00060000,...,-6.3750000,13.8170000,0.0920000,-0.0920000,12.1800000,0.0200000,-0.0200000,13.7062000,0.0002710,-0.0002710
2,Kepler-11 c,Kepler-11,1,6,Transit,2010,Kepler,0,13.02410000,0.00130000,...,-6.3750000,13.8170000,0.0920000,-0.0920000,12.1800000,0.0200000,-0.0200000,13.7062000,0.0002710,-0.0002710
3,Kepler-11 d,Kepler-11,1,6,Transit,2010,Kepler,0,22.68450000,0.00090000,...,-6.3750000,13.8170000,0.0920000,-0.0920000,12.1800000,0.0200000,-0.0200000,13.7062000,0.0002710,-0.0002710
4,Kepler-11 e,Kepler-11,1,6,Transit,2010,Kepler,0,31.99960000,0.00080000,...,-6.3750000,13.8170000,0.0920000,-0.0920000,12.1800000,0.0200000,-0.0200000,13.7062000,0.0002710,-0.0002710
5,Kepler-11 f,Kepler-11,1,6,Transit,2010,Kepler,0,46.68880000,0.00270000,...,-6.3750000,13.8170000,0.0920000,-0.0920000,12.1800000,0.0200000,-0.0200000,13.7062000,0.0002710,-0.0002710
6,Kepler-11 g,Kepler-11,1,6,Transit,2010,Kepler,0,118.38070000,0.00100000,...,-6.3750000,13.8170000,0.0920000,-0.0920000,12.1800000,0.0200000,-0.0200000,13.7062000,0.0002710,-0.0002710


In [35]:
# Defining Solar Mass
SMass = float(K11Data.iloc[1,55]) * u.Msun
# Classifying Planets
class planets:
    def __init__ (body, OrbPer, OrbMaj, OrbEcc, PRad, PMass):
        body.OrbPer = OrbPer
        body.OrbMaj = OrbMaj
        body.OrbEcc = OrbEcc
        body.PRad = PRad
        body.PMass = PMass 
    def simple_radii(a): #Equating Angular Acceleration to the Force of Gravity to find r
        M = SMass.to(u.kg) * u.kg
        x = M * c.G * (a.OrbPer.to(u.s) * u.s)**2
        y = (2*np.pi)**2
        return (x / y)**(1/3)
    def real_radii(p):
        e = p.OrbEcc
        a = p.OrbMaj 
        r = (e + 1)*a
        return r.to_value(u.m)
    def initVel(p):
        M = SMass.to(u.kg)
        m = (p.PMass).to(u.kg) 
        x = c.G.value * (M + m)
        y = ((2 / (planets.real_radii(p).value)) - (1 / p.OrbMaj.to_value(u.m)))
        return np.sqrt(x * y)
pl = []
for i in range(1, 7):
    iOrbPer = K11Data.iloc[i, 8] * u.day
    iOrbMaj = K11Data.iloc[i, 12] * u.au
    iOrbEcc = float(K11Data.iloc[i, 33])
    iPRad = K11Data.iloc[i, 16] * u.earthRad
    iPMass = K11Data.iloc[i, 24] * u.earthMass
    pl.append(planets(iOrbPer, iOrbMaj, iOrbEcc, iPRad, iPMass))
planets.real_radii(pl[2])
x = SMass.to(u.kg)
print(pl[0].PMass)
x.unit.decompose

1.9 earthMass


<bound method IrreducibleUnit.decompose of Unit("kg")>

In [25]:
"""#Accelerations from centripetal and gravitational forces
# r1 is the radius vector of the planet, v1 is its velocity
# m2 is the STAR's mass
def cent_accel (r1, v1): # where r1 and v1 are vectors already
    rad_mag = np.linalg.norm(r1)
    #vel_perp = np.cross(v1, r1) / rad_mag
    velp_mag = np.linalg.norm(v1)
    cent_uvec = -np.cross(v1, [0, 0, np.cross(v1,r1)]) / (velp_mag**2 * rad_mag) #pointing the unit vector to norm the orbital path 
    cent_accel_mag = (velp_mag)**2 / rad_mag
    RESHAPE = cent_accel_mag * cent_uvec # IM RESHAPING FROM 3D TO 2D
    cent_accel_vec = np.array([RESHAPE[0],RESHAPE[1]])
    return cent_accel_vec

def grav_accel (r1, m2): # where r1 is a vector
    rad_mag = np.linalg.norm(r1) #distance from star to planet
    rad_uvec = r1 / rad_mag
    grav_accel_mag = (c.G.value * m2) / rad_mag**2 
    grav_accel_vec = grav_accel_mag * -rad_uvec
    return grav_accel_vec

def tot_accel (r1, v1, m2):
    #c_accel = cent_accel(r1, v1)
    g_accel = grav_accel(r1, m2)
    return g_accel #+ c_accel

def OrbitEquation(w, t, m1, m2): # w is an array containing positions and velocities
    r1 = w[:2]
    
    v1 = w[2:4]
    
    acceleration = tot_accel(r1, v1, m2) # derivative of velocity (accel)
    velocity = v1 + t*acceleration # derivative of position (vel)

    derivs = np.concatenate((velocity, acceleration)) # joining the two arrays, definitely remember concatenate
    
    return derivs"""

"#Accelerations from centripetal and gravitational forces\n# r1 is the radius vector of the planet, v1 is its velocity\n# m2 is the STAR's mass\ndef cent_accel (r1, v1): # where r1 and v1 are vectors already\n    rad_mag = np.linalg.norm(r1)\n    #vel_perp = np.cross(v1, r1) / rad_mag\n    velp_mag = np.linalg.norm(v1)\n    cent_uvec = -np.cross(v1, [0, 0, np.cross(v1,r1)]) / (velp_mag**2 * rad_mag) #pointing the unit vector to norm the orbital path \n    cent_accel_mag = (velp_mag)**2 / rad_mag\n    RESHAPE = cent_accel_mag * cent_uvec # IM RESHAPING FROM 3D TO 2D\n    cent_accel_vec = np.array([RESHAPE[0],RESHAPE[1]])\n    return cent_accel_vec\n\ndef grav_accel (r1, m2): # where r1 is a vector\n    rad_mag = np.linalg.norm(r1) #distance from star to planet\n    rad_uvec = r1 / rad_mag\n    grav_accel_mag = (c.G.value * m2) / rad_mag**2 \n    grav_accel_vec = grav_accel_mag * -rad_uvec\n    return grav_accel_vec\n\ndef tot_accel (r1, v1, m2):\n    #c_accel = cent_accel(r1, v1)\n    g

In [26]:
### CHUZIDAS NEW CODE FOR MOTION ###

# the model for integration:
def modelx(X, t):
    M_s = SMass.to(u.kg)
    return[X[1], c.G.value * M_s/np.square(X[0])]
#X[0] is the x-velocity
#X[1] is the x-acceleration
def modely(Y,t):
    M_s = SMass.to(u.kg)
    return[Y[1], c.G.value * M_s/np.square(Y[0])]
#Y[0] is the y-velocity
#Y[1] is the y-acceleration



In [36]:
#INTEGRATING AND DETERMINING THE POSITIONS

# Define variables for a planet
sol = np.array([],[])
for i in range (0,1): #will change to 0,6 once I get the np.array to append properly
    """mp = pl[i].PMass.to(u.kg) # mass of planet
    ms = SMass.to(u.kg) #star
    ri = planets.simple_radii(pl[i]) / u.m
    vi = ((c.G.value*ms) / (ri))**(1/2)"""
#The differential equation of motion:
    rad0 = planets.real_radii(pl[i])
    vel0 = planets.initVel(pl[i])
    Xi = np.array((rad0, 0))
    Yi = np.array((0, vel0))
    xpos = np.array(())
    ypos = np.array(())

    step = np.linspace(0, 4e5, 200)
    xpos = scipy.integrate.odeint(modelx,[Xi[0], Xi[1]], step)
    ypos = scipy.integrate.odeint(modely,[Yi[0], Yi[1]], step)
    
    # REORIENT INTO ONE LIST
    funky = np.array[()]
    for i in range(0, (len(xpos)) / 2):
        funky = np.append(funky, xpos[i])
        funky = np.append(funky, xpos[i + 1])
        funky = np.append(funky, ypos[i])
        funky = np.append(funky, ypos[i + 1])
    
"""# INITAL position vectors 
    r1 = np.array([ri, 0])
    r2 = np.array([0, 0]) # at orgin

# INITIAL velocities of each planet
    v1 = np.array([0, vi])
    v2 = np.array([0, 0]) # stationary sun in this equation
    
# Package initial parameters into one array (just easier to work with this way)
    init_params = np.array([r1, v1])
    init_params = init_params.flatten()
    time_span = np.linspace(0, 4e5, 800)  # run for 1e6 (~12 days) with 500 points
    print("i")
# Run the ODE solver to solve for position of each planet
    funky = scipy.integrate.odeint(OrbitEquation, init_params, time_span, args=(mp,ms))
    #sol = np.append(sol, funky, axis=0) """
    
    

print(funky)

UnitConversionError: Can only apply 'add' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)

In [8]:
# Initilize writer 
metadata = dict(title='2D testing', artist='Nicooo')
writer = FFMpegWriter(fps=24, metadata=metadata, bitrate=2000)
fig = plt.figure(dpi=200)

<Figure size 1280x960 with 0 Axes>

In [9]:
r1_sol = funky[:500,:2]

# SAVE AS MP4 (will be saved in whatever directory you are working in)
fig, ax = plt.subplots()

with writer.saving(fig, "simple_orbit.mp4", dpi=200):
    for i in range(len(time_span)):

        ax.clear()

        ax.plot(r1_sol[:i,0],r1_sol[:i,1],color="brown", alpha=0.5)
        ax.scatter(r1_sol[i,0],r1_sol[i,1],color="brown",marker="o",s=20, zorder=5) # planet
        
        ax.scatter(0, 0, color="red",marker="*",s=50, zorder=5) # star
        
        ax.set_xlim(-5e10, 5e10)
        ax.set_ylim(-5e10, 5e10)
        
        plt.draw()
        plt.pause(0.008)
        writer.grab_frame()

NameError: name 'funky' is not defined

In [200]:
pl[0].PMass

Unit("1.9 earthMass")

In [108]:
(ri * 2*np.pi) / pl[0].OrbPer.to(u.s)

<Quantity 96519.66056104>

In [156]:
pl[0].PMass.to(u.kg)

1.134711894880362e+25

In [73]:
x = planets.simple_radii(pl[0])

In [74]:
x.to(u.au) # NOTE: Using the simple radii function I'm abler to get the same values from NASA! :))

<Quantity 0.09141677 AU>

In [42]:
r1 = [0,1]
v1 = [1,1]

#cent_uvec = -np.cross(v1, [0, 0, np.cross(v1,r1)])
#cent_uvec
x =cent_accel(r1, v1)
x = np.array([x[0],x[1]])
print(x)

[-2.  2.]


In [52]:
for i in range (1,6):
    e = (K11Data.iloc[i,33]*u.s).to(u.s) #eccentricity (unitless))
    Mp = (K11Data.iloc[i,25]*u.earthMass).to(u.kg) #planet mass in kg (csv in Earth masses)
    T = (K11Data.iloc[i,9]*u.d).to(u.s)
    initpos = e * np.cbrt((c.G.value*Mp*T) / (4 * np.pi)**2) #defining initial position (meters??)
    print(initpos)
    
planets.simple_radii(pl[1]) 

2555.7371781742086
2435.7406619630124
215.79832612211112
767.5716213039001
1011.5138901486864


<Quantity 1.59875538e+10 m>

In [58]:
planets.real_radii(pl[1]) * u.dimensionless_unscaled

<Quantity 1.64231534e+10 m>

In [69]:
int(rad0)


14226009514

In [68]:
int(vel0)

92481

In [10]:
K11Data.iloc[1,55]


'0.96'

In [32]:
c.G.value

6.6743e-11